In [9]:
#ALL CODE
import json
import os
from openai import OpenAI
from dotenv import load_dotenv
import time

def show_json(obj):
    display(json.loads(obj.model_dump_json()))

api_key = os.getenv("OPENAI_API_KEY")
assistent_id = os.getenv("ASSISTANT_ID")
client = OpenAI(api_key=api_key)

def create_new_thread():
    thread = client.beta.threads.create()
    return thread

def summit_message(assistent_id, thread_id, user_message, model="gpt-4o-mini"):
    #메세지 전송
    client.beta.threads.messages.create( 
        thread_id=thread_id,
        role="user",
        content=user_message
    )

    #실행시켜 assistant와 연결
    run = client.beta.threads.runs.create(
        thread_id=thread_id,
        assistant_id=assistent_id,
        model=model
    )

    return run

# run status가 queued or in_progress 상태가 아닐때 까지 지정된 시간간격으로 확인하고 완료되면 값을 반환해주는 함수
def wait_on_run(run, thread):
    while run.status=="queued" or run.status=="in_progress":
        run=client.beta.threads.runs.retrieve(
            thread_id=thread,
            run_id=run.id,
        )
        time.sleep(0.5)
    return run

# 작업 완료시 Thread값 가져오는 함수
def get_response(thread_id):
    return client.beta.threads.messages.list(thread_id=thread_id, order="asc") # 최신질문, 답변이 맨 위로 올라오게 asc 설정

def print_message(response):
    for res in response:
        print(f"[res.role.upper()]\n{res.content[0].text.value}\n")
        
# OpenAI의 JSON 값 전체 내용 확인할 때 사용
def show_json(obj):
    print(json.loads(obj.model_dump_json()))

def main():
    #Threads id 생성
    THREAD_ID = create_new_thread().id
    model = "gpt-4o-mini"
    
    while True:
        #assistant에게 전달할 질문내용
        question = input("Q: ")

        #종료 키워드
        if question.lower() in ["exit", "종료", "quit"]:
            print("종료합니다.")
            break

        #Thread에 메세지를 전송하고 Run실행
        run = summit_message(assistent_id, THREAD_ID, question, model)

        #Run상태 확인
        run1 = wait_on_run(run, THREAD_ID)

        #Run 확인 후 데이터 출력
        print_message(get_response(THREAD_ID).data[-2:])



In [10]:
main()

[res.role.upper()]
현재 로그인한 계정 이름이 뭐야?

[res.role.upper()]
현재 로그인한 계정에 대한 정보는 제공할 수 없습니다. 개인정보 보호 정책에 따라 사용자의 계정 정보에 접근할 수 없기 때문입니다. 다른 질문이 있으신가요?

[res.role.upper()]
그래. 그러면 나는 지금 Assistants를 통해 동작하고 있는데 내가 정한 Assistants의 이름은 뭐야?

[res.role.upper()]
현재 당신이 설정한 Assistant의 이름은 "Assistant"입니다. 필요하신 다른 정보를 말씀해 주세요!

[res.role.upper()]
파일을 검색하는 데 오류가 발생했습니다. 다시 시도해 보겠습니다. 잠시만 기다려 주세요.

[res.role.upper()]
파일 검색에 문제가 발생하여 파일 제목을 불러올 수 없습니다. 파일 관련하여 다른 질문이 있으시면 말씀해 주세요. 도움이 필요하신 부분에 대해 최대한 도와드리겠습니다.

종료합니다.
